# **Using aeon Distances with scikit-learn Clusterers**

This notebook demonstrates how to integrate aeon’s distance metrics with hierarchical, density-based, and spectral clustering methods from scikit-learn. While aeon primarily supports partition-based clustering algorithms, such as $k$-means and $k$-medoids, its robust distance measures can be leveraged to enable other clustering techniques using scikit-learn.

Broadly, clustering algorithms can be categorized into partition-based, hierarchical, density-based, and spectral methods. In this notebook, we focus on using aeon’s distance metrics with:
1. **Hierarchical clustering**: `AgglomerativeClustering` with `metric="precomputed"`.
2. **Density-based clustering**: `DBSCAN` and `OPTICS` with `metric="precomputed"`.
3. **Spectral clustering**: `SpectralClustering` with `affinity="precomputed"` and the inverse of the distance matrix.

To measure similarity between time series and enable clustering, we use aeon’s precomputed distance matrices. For details about distance metrics, see the [distance examples](../distances/distances.ipynb).

## **Contents**
1. **Introduction**: Overview of clustering methods and motivation for this notebook.
2. **Loading Data**: Using the `load_unit_test` dataset from aeon.
3. **Computing Distance Matrices with aeon**: Precomputing distance matrices with aeon’s distance metrics.

4. **Hierarchical Clustering**
   4.1 sklearn.cluster.AgglomerativeClustering with metric="precomputed"

5. **Density-Based Clustering**
   5.1 sklearn.cluster.DBSCAN with metric="precomputed"
   5.2 sklearn.cluster.OPTICS with metric="precomputed"

6. **Spectral Clustering**
   6.1 sklearn.cluster.SpectralClustering with affinity="precomputed"
   6.2 Using the Inverse of the Distance Matrix

## **Introduction**

While aeon primarily focuses on partition-based clustering methods, it's possible to extend its capabilities by integrating its distance metrics with scikit-learn's clustering algorithms. This approach allows us to perform hierarchical, density-based, and spectral clustering on time series data using aeon's rich set of distance measures.

## **Loading Data**

We'll begin by loading a sample dataset. For this demonstration, we'll use the `load_unit_test` dataset from aeon.

In [ ]:
# Import & load data
from aeon.datasets import load_unit_test
X_train, y_train = load_unit_test(split="train")
X_test, y_test = load_unit_test(split="test")

# For simplicity, we'll work with the training data
X = X_train
y = y_train

print(f"Data shape: {X.shape}")
print(f"Labels shape: {y.shape}")

## **Computing Distance Matrices with aeon**
Aeon provides a variety of distance measures suitable for time series data. We'll compute the distance matrix using the Dynamic Time Warping (DTW) distance as an example.

In [ ]:
from aeon.distances import pairwise_distance

# Compute the pairwise distance matrix using DTW
distance_matrix = pairwise_distance(X, metric="dtw")

print(f"Distance matrix shape: {distance_matrix.shape}")


## **Hierarchical Clustering**
Hierarchical clustering builds a hierarchy of clusters either by progressively merging or splitting existing clusters. We'll use scikit-learn's AgglomerativeClustering with the precomputed distance matrix.

In [ ]:
from sklearn.cluster import AgglomerativeClustering
import matplotlib.pyplot as plt
import numpy as np

# Perform Agglomerative Clustering
agg_clustering = AgglomerativeClustering(
    n_clusters=2, affinity="precomputed", linkage="average"
)
labels = agg_clustering.fit_predict(distance_matrix)

# Visualize the clustering results
plt.figure(figsize=(10, 6))
for label in np.unique(labels):
    plt.plot(X[labels == label].mean(axis=0), label=f"Cluster {label}")
plt.title("Hierarchical Clustering with DTW Distance")
plt.legend()
plt.show()


## **Density-Based Clustering**
Density-based clustering identifies clusters based on the density of data points in the feature space. We'll demonstrate this using scikit-learn's `DBSCAN` and `OPTICS` algorithms.

### **DBSCAN**

DBSCAN is a density-based clustering algorithm that groups data points based on their density connectivity. 
We use the `DBSCAN` algorithm from scikit-learn with a precomputed distance matrix.


In [ ]:
from sklearn.cluster import DBSCAN

# Perform DBSCAN clustering
dbscan = DBSCAN(eps=0.5, min_samples=5, metric="precomputed")
dbscan_labels = dbscan.fit_predict(distance_matrix)

# Visualize the clustering results
plt.figure(figsize=(10, 6))
for label in np.unique(dbscan_labels):
    if label == -1:
        # Noise points
        plt.plot(X[dbscan_labels == label].mean(axis=0), label="Noise", linestyle="--")
    else:
        plt.plot(X[dbscan_labels == label].mean(axis=0), label=f"Cluster {label}")
plt.title("DBSCAN Clustering with DTW Distance")
plt.legend()
plt.show()


### **OPTICS**
OPTICS is a density-based clustering algorithm similar to DBSCAN but provides better handling of varying 
densities. We use the `OPTICS` algorithm from scikit-learn with a precomputed distance matrix.

In [ ]:
from sklearn.cluster import OPTICS

# Perform OPTICS clustering
optics = OPTICS(min_samples=5, metric="precomputed")
optics_labels = optics.fit_predict(distance_matrix)

# Visualize the clustering results
plt.figure(figsize=(10, 6))
for label in np.unique(optics_labels):
    if label == -1:
        # Noise points
        plt.plot(X[optics_labels == label].mean(axis=0), label="Noise", linestyle="--")
    else:
        plt.plot(X[optics_labels == label].mean(axis=0), label=f"Cluster {label}")
plt.title("OPTICS Clustering with DTW Distance")
plt.legend()
plt.show()


## **Spectral Clustering**
Spectral clustering performs dimensionality reduction on the data before clustering in fewer dimensions. It requires a similarity matrix, so we'll convert our distance matrix accordingly.

In [ ]:
from sklearn.cluster import SpectralClustering
import numpy as np

# Ensure the distance matrix does not contain zeros on the diagonal or elsewhere
# Now adding a small constant to avoid division by zero
epsilon = 1e-10
inverse_distance_matrix = 1 /(distance_matrix + epsilon)

# Perform Spectral Clustering with affinity="precomputed"
spectral = SpectralClustering(
    n_clusters=2, affinity="precomputed", random_state=42
)
spectral_labels = spectral.fit_predict(inverse_distance_matrix)

# Visualising the clustering results
plt.figure(figsize=(10, 6))
for label in np.unique(spectral_labels):
    plt.plot(X[spectral_labels == label].mean(axis=0), label=f"Cluster {label}")
plt.title("Spectral Clustering with Inverse Distance Matrix")
plt.legend()
plt.show()


## **References**

[1] Christopher Holder, Matthew Middlehurst, and Anthony Bagnall. A Review and Evaluation of Elastic Distance Functions for Time Series Clustering, Knowledge and Information Systems. In Press (2023).

[2] Christopher Holder, David Guijo-Rubio, and Anthony Bagnall. Barycentre averaging for the move-split-merge time series distance measure. 15th International Joint Conference on Knowledge Discovery, Knowledge Engineering and Knowledge Management (2023).

[3] Kaufman, Leonard & Rousseeuw, Peter. (1986). Clustering Large Data Sets. 10.1016/B978-0-444-87877-9.50039-X.

[4] R. T. Ng and Jiawei Han. "CLARANS: a method for clustering objects spatial data mining." IEEE Transactions on Knowledge and Data Engineering vol. 14, no. 5, pp. 1003-1016, Sept.-Oct. 2002, doi: 10.1109/TKDE.2002.1033770.

[5] Paparrizos, John, and Luis Gravano. "Fast and Accurate Time-Series Clustering." ACM Transactions on Database Systems 42, no. 2 (2017): 8:1-8:49.

[6] F. Petitjean, A. Ketterlin and P. Gancarski. “A global averaging method for dynamic time warping, with applications to clustering,” Pattern Recognition, vol. 44, pp. 678-693, 2011.

Generated using [nbsphinx](https://nbsphinx.readthedocs.io/). The Jupyter notebook can be found here[here](sklearn_clustering_with_aeon_distances.html).